Declare a numba accelerated function that computes the Halton QRNG
1. The parameter $n$ is an integer of any size
2. The parameter $p$ is a prime number

In [ ]:
"""mc_parabola.ipynb"""

# Cell 01

from numba import float64, int64, vectorize


@vectorize([float64(int64, int64)], nopython=True)
def halton(n, p):
    h, f = 0, 1
    while n > 0:
        f = f / p
        h += (n % p) * f
        n = int(n / p)
    return h


print(halton(10_000, 2))

Set the `sample_area` boundaries and the number of samples (`total_dots`) to take

In [ ]:
# Cell 02

from matplotlib.patches import Rectangle

# ((x, y) anchor point, width, height)
bbox = Rectangle((-2, 0), 4, 5).get_bbox()
print(bbox)


total_dots = 40_600
print(f"{total_dots = :,}")

Take $n$ "random" samples of 2D Cartesian points $(x,y)$ using the Halton sequence
1. Scale the results so $-2\le x_{qrng}\leq 2$ and $0\le y_{qrng}\leq 5$
2. The sample area is thus $(-2...2)\times(0...5)=20$


In [ ]:
# Cell 03

import numpy as np
import pandas as pd

x = (1 - halton(np.arange(total_dots), 2)) * bbox.width + bbox.x0
y = (1 - halton(np.arange(total_dots), 3)) * bbox.height + bbox.y0

pd.DataFrame({"x": x[:5], "y": y[:5]})

Create an array $d$ containing $y_{rnd}-f(x_{rnd})$


In [ ]:
# Cell 04

d = y - (4 - x**2)

pd.DataFrame({"x": x[:5], "y": y[:5], "d": d[:5]})

Create arrays of $(x,y)$ coordinates that are "above" or "on or below" the parabola\
Here $f(x)=4-x^2$ so if $d>0$ then the sample point is "above" the curve

In [ ]:
# Cell 05

x_in = x[d <= 0.0]
y_in = y[d <= 0.0]
x_out = x[d > 0.0]
y_out = y[d > 0.0]

pd.DataFrame(
    {"x_in": x_in[:5], "y_in": y_in[:5], "x_out": x_out[:5], "y_out": y_out[:5]}
)

Display the scatter plot of the Monte Carlo estimation

In [ ]:
# Cell 06

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(x_in, y_in, color="red", marker=".", s=0.5)
plt.scatter(x_out, y_out, color="blue", marker=".", s=0.5)
plt.title("$y=-x^2+4$")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Calculate the absolute percent error in the area estimation
1. The actual/expected definite integral is $\dfrac{32}{3}=10.6666...$
2. The observed/estimated area using the Monte Carlo formulation $=20\times\dfrac{dots_{\ inside}}{dots_{\ total}}$


In [ ]:
# Cell 07

act = 32 / 3
est = (bbox.width * bbox.height) * np.count_nonzero(d <= 0) / total_dots
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")